<a href="https://colab.research.google.com/github/jie420/0025_big_data/blob/main/W05_quiz_2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 5 Quiz

This notebook contains the SQL Quiz for Week 5. Section 1 uses the New York City data we know and love. Section 2 uses building footprint data from the Google Open Buildings dataset.

INSTRUCTIONS:

Run this notebook in Google Colab. The answer to each quesiton will be a number or a string. Input these into the corresponding question on Moodle. You have 90 minutes to attempt the quiz, so if you get stuck on a question, move on.

Make sure you run all of the cells of code in order, especially the ones that already have code in them! If you run into serious problems, try clicking on the "runtime" tab above and selecting "restart session and run all".
Use the cells below for any necessary setup.

In [4]:
!pip install duckdb

In [5]:
!wget -q https://s3.amazonaws.com/s3.cleverelephant.ca/postgis-workshop-2020.zip -O postgis-workshop-2020.zip
!unzip -q postgis-workshop-2020.zip -d postgis_workshop
!ls -R postgis_workshop | head -n 80

postgis_workshop:
postgis-workshop

postgis_workshop/postgis-workshop:
data
printing

postgis_workshop/postgis-workshop/data:
2000
nyc_census_blocks.dbf
nyc_census_blocks.prj
nyc_census_blocks.shp
nyc_census_blocks.shx
nyc_census_sociodata.sql
nyc_data.backup
nyc_homicides.dbf
nyc_homicides.prj
nyc_homicides.shp
nyc_homicides.shx
nyc_neighborhoods.dbf
nyc_neighborhoods.prj
nyc_neighborhoods.shp
nyc_neighborhoods.shx
nyc_streets.dbf
nyc_streets.prj
nyc_streets.shp
nyc_streets.shx
nyc_subway_stations.dbf
nyc_subway_stations.prj
nyc_subway_stations.shp
nyc_subway_stations.shx

postgis_workshop/postgis-workshop/data/2000:
nyc_census_blocks_2000.dbf
nyc_census_blocks_2000.prj
nyc_census_blocks_2000.shp
nyc_census_blocks_2000.shx
nyc_census_sociodata_2000.sql

postgis_workshop/postgis-workshop/printing:
nyc_data_dictionary.docx
nyc_data_dictionary.pdf
postgis-workshop-exercises.docx
postgis-workshop-exercises.pdf


# Section 1



The URL below points to a zip file containing the data on New York City that we've been working with so far.

https://s3.amazonaws.com/s3.cleverelephant.ca/postgis-workshop-2020.zip

## Question 1

Create the following tables using the corresponding shapefiles.
- nyc_neighborhoods
- nyc_census_blocks
- nyc_homicides
- nyc_streets
- nyc_subway_stations

In [7]:
import duckdb
from pathlib import Path

con = duckdb.connect("nyc.duckdb")

con.execute("INSTALL spatial;")
con.execute("LOAD spatial;")

shps = list(Path("postgis_workshop").rglob("*.shp"))
len(shps), shps[:10]

targets = {
    "nyc_neighborhoods": None,
    "nyc_census_blocks": None,
    "nyc_homicides": None,
    "nyc_streets": None,
    "nyc_subway_stations": None,
}

for p in shps:
    name = p.stem.lower()
    if name in targets:
        targets[name] = str(p)

targets

for table, shp_path in targets.items():
    if shp_path is None:
        raise FileNotFoundError(f"couldn't find {table}.shp")

    con.execute(f"DROP TABLE IF EXISTS {table};")
    con.execute(f"""
        CREATE TABLE {table} AS
        SELECT * FROM ST_Read('{shp_path}');
    """)
    print("Imported:", table, "from", shp_path)

Imported: nyc_neighborhoods from postgis_workshop/postgis-workshop/data/nyc_neighborhoods.shp
Imported: nyc_census_blocks from postgis_workshop/postgis-workshop/data/nyc_census_blocks.shp
Imported: nyc_homicides from postgis_workshop/postgis-workshop/data/nyc_homicides.shp
Imported: nyc_streets from postgis_workshop/postgis-workshop/data/nyc_streets.shp
Imported: nyc_subway_stations from postgis_workshop/postgis-workshop/data/nyc_subway_stations.shp


## Question 2:
How many subway stations are there in NYC?



In [10]:
con.execute("""
SELECT COUNT(*)
FROM nyc_subway_stations;
""").fetchone()[0]

491

## Question 3:

What combination of borough and year hard the largest number of homicide victims?

In [16]:
con.execute("""
SELECT
  BORONAME,
  YEAR,
  SUM(CAST(NUM_VICTIM AS INTEGER)) AS victims
FROM nyc_homicides
GROUP BY BORONAME, YEAR
ORDER BY victims DESC
LIMIT 1;
""").df()

,BORONAME,YEAR,victims
0,Brooklyn,2003,235.0


## Question 4:

Which subway station has the largest number of shooting victims within a 300 meter radius?


In [19]:
con.execute("""
SELECT
  s.name AS station,
  SUM(CAST(h.NUM_VICTIM AS INTEGER)) AS shooting_victims_300m
FROM nyc_subway_stations AS s
JOIN nyc_homicides AS h
  ON ST_DWithin(s.geom, h.geom, 300)
WHERE h.WEAPON = 'gun'
GROUP BY s.name
ORDER BY shooting_victims_300m DESC
LIMIT 1;
""").df()

,station,shooting_victims_300m
0,Franklin Ave,28.0


## Question 5

what is the most densely populated residential street in new york? (total population of intersecting census blocks divided by the total length of the street)

In [22]:
con.execute("""
WITH residential AS (
  SELECT ID, NAME, geom
  FROM nyc_streets
  WHERE lower(TYPE) LIKE '%residential%'
),

street_block AS (
  SELECT DISTINCT
    s.NAME AS street_name,
    c.blkid AS block_id,
    c.popn_total AS popn_total
  FROM residential s
  JOIN nyc_census_blocks c
    ON ST_Intersects(s.geom, c.geom)
),

pop_by_street AS (
  SELECT
    street_name,
    SUM(popn_total) AS pop_total
  FROM street_block
  GROUP BY street_name
),

len_by_street AS (
  SELECT
    NAME AS street_name,
    SUM(ST_Length(geom)) AS street_len
  FROM residential
  GROUP BY NAME
)

SELECT
  p.street_name,
  p.pop_total,
  l.street_len,
  p.pop_total / NULLIF(l.street_len, 0) AS pop_per_meter
FROM pop_by_street p
JOIN len_by_street l USING (street_name)
ORDER BY pop_per_meter DESC
LIMIT 1;
""").df()

,street_name,pop_total,street_len,pop_per_meter
0,W 66 St,2969.0,0.920966,3223.78748


# Section 2

This section uses internet speed data from [Ookla](https://github.com/teamookla/ookla-open-data); take a minute to read the documentation of the dataset.

Create a table called ookla_nyc which reads the data from this url:

https://storage.googleapis.com/qm2/CASA0025/ookla_nyc.parquet




In [23]:
!pip install duckdb
import duckdb

con = duckdb.connect("nyc.duckdb")
con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")

con.execute("DROP TABLE IF EXISTS ookla_nyc;")
con.execute("""
CREATE TABLE ookla_nyc AS
SELECT *
FROM read_parquet('https://storage.googleapis.com/qm2/CASA0025/ookla_nyc.parquet');
""")

con.execute("SELECT COUNT(*) AS n FROM ookla_nyc;").df()

,n
0,7645


## Question 6
How many tiles are contained within new york city neighbourhoods?

In [25]:
con.execute("""
SELECT
  COUNT(DISTINCT o.quadkey) AS num_tiles
FROM ookla_nyc AS o
JOIN nyc_neighborhoods AS n
  ON ST_Intersects(
       ST_GeomFromText(o.geometry),
       n.geom
     );
""").df()

,num_tiles
0,2962


## Question 7

Which neighbourhood has the highest average download speed? use 'within' rather than intersects, and ignore tiles with fewer than 3 tests.

In [27]:
con.execute("""
SELECT
  n.NAME AS neighborhood,
  AVG(o.avg_d_kbps) AS avg_download_kbps
FROM ookla_nyc AS o
JOIN nyc_neighborhoods AS n
  ON ST_Within(
       ST_GeomFromText(o.geometry),
       n.geom
     )
WHERE o.tests >= 3
GROUP BY n.NAME
ORDER BY avg_download_kbps DESC
LIMIT 1;
""").df()

,neighborhood,avg_download_kbps
0,Woodside,455960.333333


## Question 8



What is the name of the subway station closest to the tile with the highest download speed?

In [28]:
con.execute("""
WITH fastest_tile AS (
  SELECT
    ST_GeomFromText(geometry) AS geom
  FROM ookla_nyc
  ORDER BY avg_d_kbps DESC
  LIMIT 1
)
SELECT
  s.name AS station_name,
  ST_Distance(s.geom, f.geom) AS distance
FROM nyc_subway_stations AS s
CROSS JOIN fastest_tile AS f
ORDER BY distance ASC
LIMIT 1;
""").df()

,station_name,distance
0,190th St,14157.780068


## Question 9

Which neighbourhood has the highest number of devices per capita? Use only spatial intersections.

In [29]:
con.execute("""
WITH
pop_by_neigh AS (
  SELECT
    n.name AS neighborhood,
    SUM(b.popn_total) AS pop_total
  FROM nyc_neighborhoods n
  JOIN (
    SELECT DISTINCT blkid, popn_total, geom
    FROM nyc_census_blocks
  ) b
    ON ST_Intersects(n.geom, b.geom)
  GROUP BY n.name
),

dev_by_neigh AS (
  SELECT
    n.name AS neighborhood,
    SUM(o.devices) AS devices_total
  FROM nyc_neighborhoods n
  JOIN (
    SELECT DISTINCT quadkey, devices, ST_GeomFromText(geometry) AS geom
    FROM ookla_nyc
  ) o
    ON ST_Intersects(n.geom, o.geom)
  GROUP BY n.name
)

SELECT
  d.neighborhood,
  d.devices_total,
  p.pop_total,
  d.devices_total * 1.0 / NULLIF(p.pop_total, 0) AS devices_per_capita
FROM dev_by_neigh d
JOIN pop_by_neigh p USING (neighborhood)
ORDER BY devices_per_capita DESC
LIMIT 1;
""").df()

,neighborhood,devices_total,pop_total,devices_per_capita
0,Corona,162.0,333.0,0.486486


## Question 10


What is the average upload speed for tiles that are within 200 meters of L train stops and intersect with census blocks where the black population is above 50%?

In [30]:
con.execute("""
WITH
l_stops AS (
  SELECT geom
  FROM nyc_subway_stations
  WHERE strpos(routes, 'L') > 0
),

black_blocks AS (
  SELECT geom
  FROM nyc_census_blocks
  WHERE 1.0 * popn_black / NULLIF(popn_total, 0) > 0.5
),

eligible_tiles AS (
  SELECT DISTINCT
    o.quadkey,
    o.avg_u_kbps
  FROM ookla_nyc o
  WHERE
    EXISTS (
      SELECT 1
      FROM l_stops s
      WHERE ST_DWithin(ST_GeomFromText(o.geometry), s.geom, 200)
    )
    AND EXISTS (
      SELECT 1
      FROM black_blocks b
      WHERE ST_Intersects(ST_GeomFromText(o.geometry), b.geom)
    )
)

SELECT AVG(avg_u_kbps) AS avg_upload_kbps
FROM eligible_tiles;
""").df()

,avg_upload_kbps
0,113172.961538
